In [1]:
import numpy as np
import math
import pandas as pd

from tqdm import tqdm
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.preprocessing import MinMaxScaler
from pandas.plotting import register_matplotlib_converters

import torch
from torch import nn as nn, optim as optim, functional as F
from datetime import datetime

# A 

In [2]:
sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#93D30C", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 14, 10
register_matplotlib_converters()

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [19]:
A = np.array([
    [(math.sqrt(99))/10, -1/10],
    [1/10, (math.sqrt(99))/10]
    ])

B = np.array([
    [math.sqrt(2)/2,-2/2],
    [math.sqrt(2)/2,math.sqrt(2)/2]
])


eps1 = np.random.normal(loc=0, scale=0.2, size=[2, 2000])
eps2 = np.random.normal(loc=0, scale=0.2**2, size=[2, 2000])


x0 = np.array([1,0])+eps1[:,0] #do we need to add noise to XO?
y0 = B.dot(x0) + eps2[:,0] #not true- how do w know the real YO?

length = 2000
train_size = 1000
test_size = 1000

x, y_hat = [x0], [y0]

for i in range (length-1):
  # X(k+1) = AX(k) + ϵ1(k+1)
  x_next = a.dot(x[i]) + eps1[:, i+1]
  x.append(x_next)
  #Y(k+1) = BX(k+1) + ϵ2(k+1)
  y_next = b.dot(x_next) + eps2[:, i+1]
  y_hat.append(y_next)
  i = i+1

#need to update x_hat[i] accordingly
X = pd.DataFrame(x)
y = pd.DataFrame(y_hat)

LSTM cells:


*   C*t-1*- Cell state/Long-term memory- (last time stemp output)
*   Hidden-State (Short term memmory)- H*t-1*
*   Input- Y*t-1* (last time stemp output)


INITILAZATIONS:
H0,C0- tensors of Zeroes. in the backpopagation stage the network will learn these states by vy theire deriviations according to the loss functions.

Stages:
1.  Forget gate- Values between 0 to 1=> sigmoid(W *forget* (H*t-1*,Y*t-1*)+BIAS *forget* ) * C*t-1*

2.  Input gate- - Values between -1 to 1==> sigmoid(Wi1(H*t-1*,Y*t-1*)+BIASi1) * tanjh(Wi2(H*t-1*,Y*t-1*)+BIASi2)

>>The outcome represents the information to be kept in the long-term memory and used as the output. As the layer is being trained through back-propagation, the weights in the sigmoid function will be updated such that it learns to only let the useful pass through while discarding the less critical features

3.  1+2: will define our C**t** value (New Long term memmory)

4.  sigmoid(W output1 (H*t-1*,Y*t-1*)+BIAS output1) * tan (W output1(C**t**) + BIAS output2)- will define our output (Y_hat **t**) and out new Hidden state (H_hat **t**)  
>First, the previous short-term memory and current input (H*t-1*,Y*t-1*) will be passed into a sigmoid function (Yes, this is the 3rd time we’re doing this) with different weights yet again to create the third and final filter. Then, we put the new long-term memory (C**t**) through an activation tanh function. The output from these 2 processes will be multiplied to produce the new short-term memory ( H**t**). The short-term ( H**t**) and long-term memory (C**t**) produced by these gates will then be carried over to the next cell for the process to be repeated. The output of each time step can be obtained ( Y**t**) from the short-term memory, also known as the hidden state.

#https://towardsdatascience.com/building-rnn-lstm-and-gru-for-time-series-using-pytorch-a46e5b094e7b



## Building the sets

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def train_val_test_split(X, y,val_ratio, test_ratio):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio, shuffle=False)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_ratio, shuffle=False)
    return X_train, X_val, X_test, y_train, y_val, y_test

X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(X , y, 0.2, 0.5)

scaler = MinMaxScaler()
X_train_arr = scaler.fit_transform(X_train)
X_val_arr = scaler.transform(X_val)
X_test_arr = scaler.transform(X_test)

y_train_arr = scaler.fit_transform(y_train)
y_val_arr = scaler.transform(y_val)
y_test_arr = scaler.transform(y_test)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim

input_dim = len(X_train.columns)
output_dim = len(y_train.columns)
hidden_dim = 64
layer_dim = 3
dropout = 0.2
n_epochs = 100
learning_rate = 1e-3
weight_decay = 1e-6
batch_size =50